In [1]:
from __future__ import annotations

import logging

logging.basicConfig(level=logging.INFO)

In [2]:
from nshconfig._src.root import RootConfig

print(RootConfig.__mro__)
# Find which class in the MRO is responsible for the __init__ method
init_owner = next(
    (cls for cls in RootConfig.__mro__ if "__init__" in cls.__dict__), None
)

print(
    f"__init__ method for RootConfig is defined in: {init_owner.__name__ if init_owner else 'Not found'}"
)

# Optionally, look at the actual method
if init_owner:
    print(f"\nMethod signature:")
    print(init_owner.__init__.__code__)


INFO:nshconfig._src.config:Default hash function set for class: Invalid
INFO:nshconfig._src.config:Default hash function set for class: RootConfig


(<class 'nshconfig._src.root.RootConfig'>, <class 'pydantic_settings.main.BaseSettings'>, <class 'nshconfig._src.config.Config'>, <class 'pydantic.main.BaseModel'>, <class 'collections.abc.MutableMapping'>, <class 'collections.abc.Mapping'>, <class 'collections.abc.Collection'>, <class 'collections.abc.Sized'>, <class 'collections.abc.Iterable'>, <class 'collections.abc.Container'>, <class 'object'>)
__init__ method for RootConfig is defined in: RootConfig

Method signature:
<code object __init__ at 0x2ba93f40, file "/home/nimashoghi/repositories/nshconfig/src/nshconfig/_src/root.py", line 123>


In [3]:
from typing_extensions import override

import nshconfig as C


class Init(C.Config):
    directory: C.CliPositionalArg[str]

    @override
    def cli_cmd(self) -> None:
        print(f'git init "{self.directory}"')
        # > git init "dir"
        self.directory = "ran the git init cli cmd"


class Clone(C.Config):
    repository: C.CliPositionalArg[str]
    directory: C.CliPositionalArg[str]

    @override
    def cli_cmd(self) -> None:
        print(f'git clone from "{self.repository}" into "{self.directory}"')
        self.directory = "ran the clone cli cmd"


class Git(C.Config):
    clone: C.CliSubCommand[Clone]
    init: C.CliSubCommand[Init]

    @override
    def cli_cmd(self) -> None:
        ret = self.cli_run_subcommand()
        print(f"{ret=}")


cmd = C.CliApp.run(Git, cli_args=["init", "dir"])
cmd

INFO:nshconfig._src.config:Default hash function set for class: Init
INFO:nshconfig._src.config:Default hash function set for class: Clone
INFO:nshconfig._src.config:Default hash function set for class: Git
INFO:nshconfig._src.config:Default hash function set for class: CliAppBaseSettings


git init "dir"
ret=Init(directory='ran the git init cli cmd')


Git(clone=None, init=Init(directory='ran the git init cli cmd'))

In [6]:
import sys


class Git(C.RootConfig):
    model_config = {"cli_parse_args": True}

    clone: C.CliSubCommand[Clone]
    init: C.CliSubCommand[Init]

    @override
    def cli_cmd(self) -> None:
        ret = self.cli_run_subcommand()
        print(f"{ret=}")


sys.argv = ["example.py", "init", "dir"]

instance = C.CliApp.run(Git, cli_args=["init", "dir"])
print(instance)
Git()

INFO:nshconfig._src.config:Default hash function set for class: Git
INFO:nshconfig._src.root:BaseSettings.__init__ called from outside pydantic_settings, skipping BaseSettings magic initialization


git init "dir"
ret=Init(directory='ran the git init cli cmd')
clone=None init=Init(directory='ran the git init cli cmd')


ValidationError: 2 validation errors for Git
clone
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
init
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [12]:
Git(clone=None, init=None).cli_active_subcommand() is None

INFO:nshconfig._src.root:BaseSettings.__init__ called from outside pydantic_settings, skipping BaseSettings magic initialization


True